In [2]:
# Import necessary libraries
import pandas as pd
import numpy as np
from scipy import spatial
from IPython.display import display
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from rake_nltk import Rake
import nltk
import re

In [2]:
# Load the Hindi movies dataset
movies_hindi = pd.read_csv("data/bollywood_full_1950-2019.csv")

In [3]:
# Select relevant columns and rename them
movies_hindi = movies_hindi[["imdb_id", 'original_title', "genres", "story", "actors"]]
movies_hindi.rename(columns= {"actors": "cast", "imdb_id": "id"}, inplace=True)

In [4]:
movies_hindi

,id,original_title,genres,story,cast
0,tt8291224,Uri: The Surgical Strike,Action|Drama|War,Divided over five chapters the film chronicle...,Vicky Kaushal|Paresh Rawal|Mohit Raina|Yami Ga...
1,tt9472208,Battalion 609,War,The story revolves around a cricket match betw...,Vicky Ahuja|Shoaib Ibrahim|Shrikant Kamat|Elen...
2,tt6986710,The Accidental Prime Minister,Biography|Drama,Based on the memoir by Indian policy analyst S...,Anupam Kher|Akshaye Khanna|Aahana Kumra|Atul S...
3,tt8108208,Why Cheat India,Crime|Drama,The movie focuses on existing malpractices in ...,Emraan Hashmi|Shreya Dhanwanthary|Snighdadeep ...
4,tt6028796,Evening Shadows,Drama,While gay rights and marriage equality has bee...,Mona Ambegaonkar|Ananth Narayan Mahadevan|Deva...
...,...,...,...,...,...
4325,tt0268614,Samadhi,Drama,The story is based on the true incident at INA...,Ashok Kumar|Nalini Jaywant|Kuldip Kaur|Shyam|M...
4326,tt0244182,Sangram,Drama,After the death of his wife a policeman fails...,Ashok Kumar|Nalini Jaywant|Nawab|Sajjan|Tiwari...
4327,tt0269826,Sargam,Drama|Family,NaN,Raj Kapoor|Rehana|Om Prakash|David Abraham|Rad...
4328,tt0243555,Sheesh Mahal,Drama,Thakur Jaspal Singh lives in the prestigious a...,Sohrab Modi|Naseem Banu|Pushpa Hans|Nigar Sult...


In [5]:
# Load additional datasets for crew information
movies_hindi_makers = pd.read_csv("data/bollywood_crew_1950-2019.csv")
movies_hindi_crew_data = pd.read_csv("data/bollywood_crew_data_1950-2019.csv")
movies_hindi_writers_data = pd.read_csv("data/bollywood_writers_data_1950-2019.csv")

In [6]:
movies_hindi_makers.head()

,imdb_id,directors,writers
0,tt0042184,nm0025608,nm0025608|nm0324690
1,tt0042207,nm0490178,nm0161032|nm1879927
2,tt0042225,nm0707533,\N
3,tt0042233,nm0788880,nm0592578|nm0788880
4,tt0042380,nm0439074,nm1278450|nm0438022|nm1301772


In [8]:
# Filter out movies with missing director or writer information
movies_hindi_makers = movies_hindi_makers[(movies_hindi_makers["directors"] != "\\N") & (movies_hindi_makers["writers"] != "\\N")]

# Merge the datasets to combine movie and crew information
movies_hindi = movies_hindi.merge(movies_hindi_makers, how="left", left_on="id", right_on="imdb_id")

In [11]:
# Drop unnecessary columns
movies_hindi.drop(["imdb_id"], inplace=True, axis=1)
movies_hindi

,id,original_title,genres,story,cast,directors,writers
0,tt8291224,Uri: The Surgical Strike,Action|Drama|War,Divided over five chapters the film chronicle...,Vicky Kaushal|Paresh Rawal|Mohit Raina|Yami Ga...,nm2336554,nm2336554
1,tt9472208,Battalion 609,War,The story revolves around a cricket match betw...,Vicky Ahuja|Shoaib Ibrahim|Shrikant Kamat|Elen...,nm10342614,nm2366632
2,tt6986710,The Accidental Prime Minister,Biography|Drama,Based on the memoir by Indian policy analyst S...,Anupam Kher|Akshaye Khanna|Aahana Kumra|Atul S...,nm9049092,nm9049093|nm10376880|nm9049092|nm4577451|nm103...
3,tt8108208,Why Cheat India,Crime|Drama,The movie focuses on existing malpractices in ...,Emraan Hashmi|Shreya Dhanwanthary|Snighdadeep ...,nm2265536,nm8006178
4,tt6028796,Evening Shadows,Drama,While gay rights and marriage equality has bee...,Mona Ambegaonkar|Ananth Narayan Mahadevan|Deva...,nm1482161,nm1481242|nm1482161
...,...,...,...,...,...,...,...
4417,tt0268614,Samadhi,Drama,The story is based on the true incident at INA...,Ashok Kumar|Nalini Jaywant|Kuldip Kaur|Shyam|M...,NaN,NaN
4418,tt0244182,Sangram,Drama,After the death of his wife a policeman fails...,Ashok Kumar|Nalini Jaywant|Nawab|Sajjan|Tiwari...,nm0611527,nm0310170|nm0611527
4419,tt0269826,Sargam,Drama|Family,NaN,Raj Kapoor|Rehana|Om Prakash|David Abraham|Rad...,NaN,NaN
4420,tt0243555,Sheesh Mahal,Drama,Thakur Jaspal Singh lives in the prestigious a...,Sohrab Modi|Naseem Banu|Pushpa Hans|Nigar Sult...,NaN,NaN


In [12]:
# Split string columns into lists
movies_hindi["genres"] = movies_hindi["genres"].str.split("|")
movies_hindi["cast"] = movies_hindi["cast"].str.split("|")
movies_hindi["directors"] = movies_hindi["directors"].str.split("|")
movies_hindi["writers"] = movies_hindi["writers"].str.split("|")

In [13]:
movies_hindi.isna().sum()

id                  0
original_title      0
genres              0
story             267
cast               10
directors         846
writers           846
dtype: int64

In [14]:
# Drop rows with missing values and reset index
movies_hindi = movies_hindi.dropna().reset_index()
movies_hindi.head()

,index,id,original_title,genres,story,cast,directors,writers
0,0,tt8291224,Uri: The Surgical Strike,"[Action, Drama, War]",Divided over five chapters the film chronicle...,"[Vicky Kaushal, Paresh Rawal, Mohit Raina, Yam...",[nm2336554],[nm2336554]
1,1,tt9472208,Battalion 609,[War],The story revolves around a cricket match betw...,"[Vicky Ahuja, Shoaib Ibrahim, Shrikant Kamat, ...",[nm10342614],[nm2366632]
2,2,tt6986710,The Accidental Prime Minister,"[Biography, Drama]",Based on the memoir by Indian policy analyst S...,"[Anupam Kher, Akshaye Khanna, Aahana Kumra, At...",[nm9049092],"[nm9049093, nm10376880, nm9049092, nm4577451, ..."
3,3,tt8108208,Why Cheat India,"[Crime, Drama]",The movie focuses on existing malpractices in ...,"[Emraan Hashmi, Shreya Dhanwanthary, Snighdade...",[nm2265536],[nm8006178]
4,4,tt6028796,Evening Shadows,[Drama],While gay rights and marriage equality has bee...,"[Mona Ambegaonkar, Ananth Narayan Mahadevan, D...",[nm1482161],"[nm1481242, nm1482161]"


In [15]:
# Map writer and director IDs to their names
def map_writers(writer_list):
    return [movies_hindi_writers_data[movies_hindi_writers_data['crew_id'] == writer_id]['name'].values[0] for writer_id in writer_list]

def map_directors(directors_list):
    return [movies_hindi_crew_data[movies_hindi_crew_data['crew_id'] == director_id]['name'].values[0] for director_id in directors_list]

movies_hindi["director"] = movies_hindi["directors"].apply(map_directors)
movies_hindi["screenplay"] = movies_hindi["writers"].apply(map_writers)

movies_hindi.drop(["directors", "writers"], axis=1, inplace=True)
movies_hindi.head(10)

,index,id,original_title,genres,story,cast,director,screenplay
0,0,tt8291224,Uri: The Surgical Strike,"[Action, Drama, War]",Divided over five chapters the film chronicle...,"[Vicky Kaushal, Paresh Rawal, Mohit Raina, Yam...",[Aditya Dhar],[Aditya Dhar]
1,1,tt9472208,Battalion 609,[War],The story revolves around a cricket match betw...,"[Vicky Ahuja, Shoaib Ibrahim, Shrikant Kamat, ...",[Brijesh Batuknath Tripathi],[Bunty Rathore]
2,2,tt6986710,The Accidental Prime Minister,"[Biography, Drama]",Based on the memoir by Indian policy analyst S...,"[Anupam Kher, Akshaye Khanna, Aahana Kumra, At...",[Vijay Ratnakar Gutte],"[Sanjaya Baru, Karl Dunne, Vijay Ratnakar Gutt..."
3,3,tt8108208,Why Cheat India,"[Crime, Drama]",The movie focuses on existing malpractices in ...,"[Emraan Hashmi, Shreya Dhanwanthary, Snighdade...",[Soumik Sen],[Mishkka Shekhawat]
4,4,tt6028796,Evening Shadows,[Drama],While gay rights and marriage equality has bee...,"[Mona Ambegaonkar, Ananth Narayan Mahadevan, D...",[Sridhar Rangayan],"[Saagar Gupta, Sridhar Rangayan]"
5,5,tt6078866,Soni,[Drama],Soni a young policewoman in Delhi and her su...,"[Geetika Vidya Ohlyan, Saloni Batra, Vikas Shu...",[Ivan Ayr],"[Ivan Ayr, Kislay Kislay]"
6,6,tt5013008,Fraud Saiyyan,"[Comedy, Drama]",Fraud Saiyyan is the story of a con artist in ...,"[Arshad Warsi, Saurabh Shukla, Flora Saini, Sa...",[Sourabh Shrivastava],"[Amal Donwaar, Prakash Jha, Sharad Tripathi]"
7,7,tt4971258,Bombairiya,"[Comedy, Crime, Drama]",It follows the story of Meghna who gets embro...,"[Radhika Apte, Akshay Oberoi, Siddhanth Kapoor...",[Pia Sukanya],"[Michael E. Ward, Pia Sukanya, Aarti S. Bagdi]"
8,8,tt6903440,Manikarnika: The Queen of Jhansi,"[Action, Biography, Drama]",Manikarnika born in Varanasi when Dixt a minis...,"[Kangana Ranaut, Rimi Sen, Atul Kulkarni, Naln...","[Radha Krishna Jagarlamudi, Kangana Ranaut]","[Vijayendra Prasad, Prasoon Joshi, Himanshu Tr..."
9,9,tt7777196,Thackeray,"[Biography, Drama]",Balasaheb Thackrey works as a cartoonist for a...,"[Nawazuddin Siddiqui, Amrita Rao, Abdul Quadir...",[Abhijit Panse],[Abhijit Panse]


In [ ]:
#  Vectorize and transform the movie stories using TF-IDF
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

cv = CountVectorizer(stop_words="english", max_df=0.9, min_df=2)
word_counts = cv.fit_transform(movies_hindi["story"])
print(word_counts)

feautres = cv.get_feature_names_out()
print(feautres)

transformer = TfidfTransformer()
transformer.fit(word_counts)

In [ ]:
count_vectors = cv.transform(movies_hindi["story"])
tfidf_vectors = transformer.transform(count_vectors).tocoo()

In [ ]:
# Create a DataFrame to store TF-IDF information
tuples = zip(tfidf_vectors.row, tfidf_vectors.col, tfidf_vectors.data)
tfidf_vectors = sorted(tuples, key=lambda x: (x[0], -x[2]))
tf_idf_df = pd.DataFrame(tfidf_vectors, columns=["movie_idx", "word_idx", "tfidf"])
tf_idf_df.head(10)

In [ ]:
# Group by movie_idx and aggregate word_idx and tfidf into lists
tf_idf_df = tf_idf_df.groupby("movie_idx")[["word_idx", "tfidf"]].agg(list).reset_index()

# Keep only the top 10 keywords for each movie
tf_idf_df["word_idx"] = tf_idf_df["word_idx"].apply(lambda x: x[:10])
tf_idf_df["tfidf"] = tf_idf_df["tfidf"].apply(lambda x: x[:10])

# Map word_idx to feature names
tf_idf_df["keywords"] = tf_idf_df["word_idx"].apply(lambda x: [feautres[word_id] for word_id in x])

In [ ]:
# Merge TF-IDF information back to the movies_hindi DataFrame
movies_hindi.merge(tf_idf_df[["movie_idx", "keywords"]], how="left", left_index=True, right_index=True)

In [18]:
# Use Rake to extract keywords from movie stories
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\91916\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\91916\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
r = Rake()

def get_keywords(story):
    r.extract_keywords_from_text(story)
    keywordList           = []
    rankedList            = r.get_ranked_phrases_with_scores()
    for keyword in rankedList:
        keyword_updated       = keyword[1].split()
        keyword_updated_string    = " ".join(keyword_updated[:2])
        keywordList.append(keyword_updated_string)
        if(len(keywordList)>5):
            break
    return keywordList

movies_hindi["keywords"] = movies_hindi["story"].apply(get_keywords)

In [30]:
# Group by movie_id and keep only the first row for each movie
movies_hindi = movies_hindi.groupby("id").first().reset_index()
movies_hindi.drop(columns=["index"], inplace=True)
movies_hindi

In [33]:
# Save cleaned Hindi movies DataFrame to a CSV file
movies_hindi.to_csv("data/all_hindi_movies_cleaned.csv", sep=",", index=True, encoding="utf-8")

In [35]:
# Clean up column names and remove prefix from IMDb IDs
movies_hindi.rename(columns={"imdb_id": "id"})
movies_hindi["id"] = movies_hindi["id"].apply(lambda x: x[2:])

In [36]:
# Load the English movies datasets
movies = pd.read_csv("data/tmdb_5000_movies.csv")
credits = pd.read_csv("data/tmdb_5000_credits.csv")

In [37]:
pd.set_option("display.max_colwidth",80)

In [38]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"": 2964, ""name"": ""future""}, {""id...",en,Avatar,"In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora o...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289}, {""name"": ""Twentieth Century...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}, {""iso_3166_1"": ""G...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso_639_1"": ""es"", ""name"": ""Espa\u...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""name"": ""Fantasy""}, {""id"": 28, ...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""name"": ""drug abuse""}, {""id"": 911...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, has come back to life and is hea...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""name"": ""Jerry Bruckheimer Film...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 80, ""...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name"": ""based on novel""}, {""id"": 4...",en,Spectre,A cryptic message from Bond’s past sends him on a trail to uncover a siniste...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""name"": ""Danjaq"", ""id"": 10761}, {""...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""}, {""iso_3166_1"": ""US"", ""name""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""}, {""iso_639_1"": ""en"", ""name"": ""...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""name"": ""Crime""}, {""id"": 18, ""name...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853, ""name"": ""crime fighter""}, {""i...",en,The Dark Knight Rises,"Following the death of District Attorney Harvey Dent, Batman assumes respons...",112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""name"": ""Warner Bros."", ""id"": 6...","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""name"": ""Adventure""}, {""id"": 878, ...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"": 839, ""name"": ""mars""}, {""id"": ...",en,John Carter,"John Carter is a war-weary, former military captain who's inexplicably trans...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States of America""}]",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [39]:
movies.shape

(4803, 20)

In [40]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""credit_id"": ""5602a8a7c3a368553...","[{""credit_id"": ""52fe48009251416c750aca23"", ""department"": ""Editing"", ""gender""..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Sparrow"", ""credit_id"": ""52fe4232c...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""department"": ""Camera"", ""gender"":..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""credit_id"": ""52fe4d22c3a368484e1...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""department"": ""Sound"", ""gender"": ..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Batman"", ""credit_id"": ""52fe4781c...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""department"": ""Sound"", ""gender"": ..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""credit_id"": ""52fe479ac3a36847f8...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""department"": ""Writing"", ""gender""..."


In [41]:
credits.shape

(4803, 4)

In [42]:
#changing the genres column from json to string
movies["genres"] = movies["genres"].apply(json.loads)
print(movies["genres"])

0       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 14, '...
1       [{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 28, ...
2       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 80, '...
3       [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 18, 'name...
4       [{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 878, ...
                                             ...                                       
4798    [{'id': 28, 'name': 'Action'}, {'id': 80, 'name': 'Crime'}, {'id': 53, 'name...
4799                   [{'id': 35, 'name': 'Comedy'}, {'id': 10749, 'name': 'Romance'}]
4800    [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'n...
4801                                                                                 []
4802                                                [{'id': 99, 'name': 'Documentary'}]
Name: genres, Length: 4803, dtyp

In [43]:
# Extract genre names from the genres column
for index,i in zip(movies.index,movies["genres"]):
    list1 = []
    for j in range(len(i)):
        list1.append(i[j]["name"])
    movies.loc[index,"genres"] = str(list1)
    

In [45]:
# Extract and clean keywords, production_companies, cast, and crew information
movies["keywords"] = movies["keywords"].apply(json.loads)

for index,i in zip(movies.index,movies["keywords"]):
    list2 = []
    
    for j in range(len(i)):
        list2.append(i[j]["name"])
        
    movies.loc[index,"keywords"] = str(list2)

movies["production_companies"] = movies["production_companies"].apply(json.loads)

for index,i in zip(movies.index,movies["production_companies"]):
    list3 = []
    
    for j in range(len(i)):
        list3.append(i[j]["name"])
        
    movies.loc[index,"production_companies"] = str(list3)
   
credits["cast"] = credits["cast"].apply(json.loads)

for index,i in zip(credits.index,credits["cast"]):
    list4 = []
    
    for j in range(len(i)):
        list4.append(i[j]["name"])
        
    credits.loc[index,"cast"] = str(list4)

    
    
credits["crew"] = credits["crew"].apply(json.loads)

for index,i in zip(credits.index,credits["crew"]):
    list4 = []
    director = "None"
    screenplay = "None"
    
    for j in range(len(i)):
        if(i[j]["job"] == "Director"):
            director = i[j]["name"]
        if(i[j]["job"] == "Screenplay"):
            screenplay = i[j]["name"]
        list4.append(i[j]["name"])
        
    credits.loc[index,"crew"] = str(list4)
    credits.loc[index,"director"] = director
    credits.loc[index,"screenplay"] = screenplay

In [46]:
credits[["cast","crew","director","screenplay"]]

,cast,crew,director,screenplay
0,"['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Stephen Lang', 'Mich...","['Stephen E. Rivkin', 'Rick Carter', 'Christopher Boyes', 'Christopher Boyes...",James Cameron,James Cameron
1,"['Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Stellan Skarsgård', 'Ch...","['Dariusz Wolski', 'Gore Verbinski', 'Jerry Bruckheimer', 'Ted Elliott', 'Te...",Gore Verbinski,Terry Rossio
2,"['Daniel Craig', 'Christoph Waltz', 'Léa Seydoux', 'Ralph Fiennes', 'Monica ...","['Thomas Newman', 'Sam Mendes', 'Anna Pinnock', 'John Logan', 'John Logan', ...",Sam Mendes,Jez Butterworth
3,"['Christian Bale', 'Michael Caine', 'Gary Oldman', 'Anne Hathaway', 'Tom Har...","['Hans Zimmer', 'Charles Roven', 'Christopher Nolan', 'Christopher Nolan', '...",Christopher Nolan,Jonathan Nolan
4,"['Taylor Kitsch', 'Lynn Collins', 'Samantha Morton', 'Willem Dafoe', 'Thomas...","['Andrew Stanton', 'Andrew Stanton', 'John Lasseter', 'Colin Wilson', 'Gail ...",Andrew Stanton,Mark Andrews
...,...,...,...,...
4798,"['Carlos Gallardo', 'Jaime de Hoyos', 'Peter Marquardt', 'Reinol Martinez', ...","['Robert Rodriguez', 'Robert Rodriguez', 'Robert Rodriguez', 'Robert Rodrigu...",Robert Rodriguez,None
4799,"['Edward Burns', 'Kerry Bishé', 'Marsha Dietlein', 'Caitlin Fitzgerald', 'Da...","['Edward Burns', 'Edward Burns', 'Edward Burns', 'William Rexer', 'William R...",Edward Burns,None
4800,"['Eric Mabius', 'Kristin Booth', 'Crystal Lowe', 'Geoff Gustafson', 'Benjami...","['Carla Hetland', 'Harvey Kahn', 'Adam Sliwinski', 'Martha Williamson', 'Mar...",Scott Smith,None
4801,"['Daniel Henney', 'Eliza Coupe', 'Bill Paxton', 'Alan Ruck', 'Zhu Shimao']","['Daniel Hsia', 'Daniel Hsia']",Daniel Hsia,None


In [47]:
#Merging both csvs into one movies df with only required features
movies = movies.merge(credits,left_on = "id",right_on = "movie_id",how = "left")
movies = movies[['id', 'original_title', 'genres', 'cast', 'director','screenplay','keywords']]

In [48]:
movies

,id,original_title,genres,cast,director,screenplay,keywords
0,19995,Avatar,"['Action', 'Adventure', 'Fantasy', 'Science Fiction']","['Sam Worthington', 'Zoe Saldana', 'Sigourney Weaver', 'Stephen Lang', 'Mich...",James Cameron,James Cameron,"['culture clash', 'future', 'space war', 'space colony', 'society', 'space t..."
1,285,Pirates of the Caribbean: At World's End,"['Adventure', 'Fantasy', 'Action']","['Johnny Depp', 'Orlando Bloom', 'Keira Knightley', 'Stellan Skarsgård', 'Ch...",Gore Verbinski,Terry Rossio,"['ocean', 'drug abuse', 'exotic island', 'east india trading company', ""love..."
2,206647,Spectre,"['Action', 'Adventure', 'Crime']","['Daniel Craig', 'Christoph Waltz', 'Léa Seydoux', 'Ralph Fiennes', 'Monica ...",Sam Mendes,Jez Butterworth,"['spy', 'based on novel', 'secret agent', 'sequel', 'mi6', 'british secret s..."
3,49026,The Dark Knight Rises,"['Action', 'Crime', 'Drama', 'Thriller']","['Christian Bale', 'Michael Caine', 'Gary Oldman', 'Anne Hathaway', 'Tom Har...",Christopher Nolan,Jonathan Nolan,"['dc comics', 'crime fighter', 'terrorist', 'secret identity', 'burglar', 'h..."
4,49529,John Carter,"['Action', 'Adventure', 'Science Fiction']","['Taylor Kitsch', 'Lynn Collins', 'Samantha Morton', 'Willem Dafoe', 'Thomas...",Andrew Stanton,Mark Andrews,"['based on novel', 'mars', 'medallion', 'space travel', 'princess', 'alien',..."
...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"['Action', 'Crime', 'Thriller']","['Carlos Gallardo', 'Jaime de Hoyos', 'Peter Marquardt', 'Reinol Martinez', ...",Robert Rodriguez,None,"['united states–mexico barrier', 'legs', 'arms', 'paper knife', 'guitar case']"
4799,72766,Newlyweds,"['Comedy', 'Romance']","['Edward Burns', 'Kerry Bishé', 'Marsha Dietlein', 'Caitlin Fitzgerald', 'Da...",Edward Burns,None,[]
4800,231617,"Signed, Sealed, Delivered","['Comedy', 'Drama', 'Romance', 'TV Movie']","['Eric Mabius', 'Kristin Booth', 'Crystal Lowe', 'Geoff Gustafson', 'Benjami...",Scott Smith,None,"['date', 'love at first sight', 'narration', 'investigation', 'team', 'posta..."
4801,126186,Shanghai Calling,[],"['Daniel Henney', 'Eliza Coupe', 'Bill Paxton', 'Alan Ruck', 'Zhu Shimao']",Daniel Hsia,None,[]


In [49]:
# Remove rows with missing values for director or screenplay
movies = movies[(movies["director"]!='None') & (movies["screenplay"]!='None')]
movies["director"] = movies["director"].apply(lambda x: [x])
movies["screenplay"] = movies["screenplay"].apply(lambda x: [x])
movies.reset_index(drop = True,inplace = True)

C:\Users\91916\AppData\Local\Temp\ipykernel_17852\2184795925.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["director"] = movies["director"].apply(lambda x: [x])
C:\Users\91916\AppData\Local\Temp\ipykernel_17852\2184795925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["screenplay"] = movies["screenplay"].apply(lambda x: [x])


In [50]:
#Cleaning up genres column
movies["genres"] = movies.loc[:, "genres"].str.strip("[]").str.replace(' ','').str.replace("'",'')
movies["genres"] = movies.loc[:, "genres"].str.split(",")

#Doing the same with cast but only taking the top 5 cast since others are not relevant
movies["cast"] = movies.loc[:, "cast"].str.strip("[]").str.replace(" ","").str.replace("'","")
movies["cast"] = movies.loc[:, "cast"].str.split(",")
movies["cast"] = movies.loc[:, "cast"].apply(lambda x: x[:5])

#Same with keywords
movies["keywords"] = movies.loc[:, "keywords"].str.strip("[]").str.replace(" ","").str.replace("'","").str.split(",")

C:\Users\91916\AppData\Local\Temp\ipykernel_17852\1930950164.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["genres"] = movies.loc[:, "genres"].str.strip("[]").str.replace(' ','').str.replace("'",'')
C:\Users\91916\AppData\Local\Temp\ipykernel_17852\1930950164.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies["genres"] = movies.loc[:, "genres"].str.split(",")
C:\Users\91916\AppData\Local\Temp\ipykernel_17852\1930950164.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [51]:
# Combining hindi and english language dataframes into a single dataframe
movies = pd.concat([movies, movies_hindi[["id", "original_title", "genres", "cast", "director", "screenplay", "keywords"]]])
movies.reset_index(drop=True, inplace=True)

In [52]:
movies

,id,original_title,genres,cast,director,screenplay,keywords
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver, StephenLang, MichelleRodriguez]",[James Cameron],[James Cameron],"[cultureclash, future, spacewar, spacecolony, society, spacetravel, futurist..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley, StellanSkarsgård, ChowYun-fat]",[Gore Verbinski],[Terry Rossio],"[ocean, drugabuse, exoticisland, eastindiatradingcompany, ""loveofoneslife"", ..."
2,206647,Spectre,"[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux, RalphFiennes, MonicaBellucci]",[Sam Mendes],[Jez Butterworth],"[spy, basedonnovel, secretagent, sequel, mi6, britishsecretservice, unitedki..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman, AnneHathaway, TomHardy]",[Christopher Nolan],[Jonathan Nolan],"[dccomics, crimefighter, terrorist, secretidentity, burglar, hostagedrama, t..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton, WillemDafoe, ThomasHadenChurch]",[Andrew Stanton],[Mark Andrews],"[basedonnovel, mars, medallion, spacetravel, princess, alien, steampunk, mar..."
...,...,...,...,...,...,...,...
6269,9495690,Pagalpanti,"[Action, Comedy]","[Kriti Kharbanda, John Abraham, Ileana D'Cruz, Anil Kapoor, Urvashi Rautela,...",[Anees Bazmee],"[Anees Bazmee, Shreya Dev Verma]","[tourist group, patriotic mission, india sets, end well, vacation, turn]"
6270,9496212,22 Yards,[Sport],"[Barun Sobti, Rajit Kapur, Panchhi Bora, Kartikey Tripathi, ]",[Mitali Ghoshal],[Samrat],"[fallen cricket, young cricketer, victorious tale, dramatic portrayal]"
6271,9558612,PM Narendra Modi,"[Biography, Drama]","[Boman Irani, Vivek Oberoi, Barkha Bisht, Darshan Kumaar, Zarina Wahab, Mano...",[Omung Kumar],"[Anirudh Chawla, Harsh Limbachiyaa, Vivek Oberoi, Sandip Ssingh]","[prime minister, india, biography]"
6272,9680136,Pati Patni Aur Woh,"[Comedy, Romance]","[Kartik Aaryan, Bhumi Pednekar, Ananya Panday, Aparshakti Khurana, ]",[Mudassar Aziz],"[Mudassar Aziz, B.R. Chopra, Jasmeet K. Reen]","[ordinary middle, chintu tyagi, another woman, wife, torn, finds]"


In [53]:
#To get each column present in terms of binary info
def to_binary(vals,all_vals):
    binary_list = []
    for i in all_vals:
        if i in vals:
            binary_list.append(1)
        else:
            binary_list.append(0)
            
    return binary_list

In [54]:
#To get all the unique genres
genrelist = []
for i in movies["genres"]:
    for j in i:
        if j not in genrelist:
            genrelist.append(j)
            
castlist = []
for i in movies["cast"]:
    for j in i:
        if j not in castlist:
            castlist.append(j)

keywordslist = []
for i in movies["keywords"]:
    for j in i:
        if j not in keywordslist:
            keywordslist.append(j)

directorsList = []
for i in movies["director"]:
    for j in i:
        if j not in directorsList:
            directorsList.append(j)

screenplayList = []
for i in movies["screenplay"]:
    for j in i:
        if j not in screenplayList:
            screenplayList.append(j)

In [55]:
movies["genres_bin"] = movies["genres"].apply(lambda x: to_binary(x,genrelist))
movies["cast_bin"] = movies["cast"].apply(lambda x: to_binary(x,castlist))
movies["director_bin"] = movies["director"].apply(lambda x: to_binary(x,directorsList))
movies["screenplay_bin"] = movies["screenplay"].apply(lambda x: to_binary(x,screenplayList))
movies["keywords_bin"] = movies["keywords"].apply(lambda x: to_binary(x,keywordslist))

In [56]:
movies

,id,original_title,genres,cast,director,screenplay,keywords,genres_bin,cast_bin,director_bin,screenplay_bin,keywords_bin
0,19995,Avatar,"[Action, Adventure, Fantasy, ScienceFiction]","[SamWorthington, ZoeSaldana, SigourneyWeaver, StephenLang, MichelleRodriguez]",[James Cameron],[James Cameron],"[cultureclash, future, spacewar, spacecolony, society, spacetravel, futurist...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ..."
1,285,Pirates of the Caribbean: At World's End,"[Adventure, Fantasy, Action]","[JohnnyDepp, OrlandoBloom, KeiraKnightley, StellanSkarsgård, ChowYun-fat]",[Gore Verbinski],[Terry Rossio],"[ocean, drugabuse, exoticisland, eastindiatradingcompany, ""loveofoneslife"", ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, ..."
2,206647,Spectre,"[Action, Adventure, Crime]","[DanielCraig, ChristophWaltz, LéaSeydoux, RalphFiennes, MonicaBellucci]",[Sam Mendes],[Jez Butterworth],"[spy, basedonnovel, secretagent, sequel, mi6, britishsecretservice, unitedki...","[1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,49026,The Dark Knight Rises,"[Action, Crime, Drama, Thriller]","[ChristianBale, MichaelCaine, GaryOldman, AnneHathaway, TomHardy]",[Christopher Nolan],[Jonathan Nolan],"[dccomics, crimefighter, terrorist, secretidentity, burglar, hostagedrama, t...","[1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,49529,John Carter,"[Action, Adventure, ScienceFiction]","[TaylorKitsch, LynnCollins, SamanthaMorton, WillemDafoe, ThomasHadenChurch]",[Andrew Stanton],[Mark Andrews],"[basedonnovel, mars, medallion, spacetravel, princess, alien, steampunk, mar...","[1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...
6269,9495690,Pagalpanti,"[Action, Comedy]","[Kriti Kharbanda, John Abraham, Ileana D'Cruz, Anil Kapoor, Urvashi Rautela,...",[Anees Bazmee],"[Anees Bazmee, Shreya Dev Verma]","[tourist group, patriotic mission, india sets, end well, vacation, turn]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 

In [57]:
movies[["id", "original_title", "genres", "cast", "director", "screenplay", "keywords"]].to_csv("data/all_movies.csv", sep=',', index=False, encoding="utf-8")

In [58]:
#Finding Similarity between two movie
def Similarity(movieID1,movieID2):
    a = movies.iloc[movieID1]
    b = movies.iloc[movieID2]
    
    genresA = a["genres_bin"]
    genresB = b["genres_bin"]
    
    genreDistance = spatial.distance.cosine(genresA,genresB)
    
    castA = a["cast_bin"]
    castB = b["cast_bin"]
    
    castDistance = spatial.distance.cosine(castA,castB)
    
    directorA = a["director_bin"]
    directorB = b["director_bin"]
    
    directorDistance = spatial.distance.cosine(directorA,directorB)
    
    screenplayA = a["screenplay_bin"]
    screenplayB = b["screenplay_bin"]
    
    screenplayDistance = spatial.distance.cosine(screenplayA,screenplayB)
    
    keywordsA = a["keywords_bin"]
    keywordsB = b["keywords_bin"]
    
    keywordsDistance = spatial.distance.cosine(keywordsA,keywordsB)
    
    #The more the distance the less similar the two movies are
    return genreDistance+castDistance+directorDistance+screenplayDistance+keywordsDistance

In [60]:
movies["id"] = pd.Series(movies.index)

In [62]:
#Gets K nearest Neighbours of baseMovie
import operator
def getNeighbours(k,basemovie):
    basemovie = movies.iloc[movies[movies["original_title"].str.contains(basemovie)].original_title.str.len().sort_values().index[0]].to_frame().T
    print("Selected Movie : "+basemovie["original_title"].values[0])
    distances = []
    for index,row in movies.iterrows():
        if row["id"]!=basemovie["id"].values[0]:
            distances.append((row["id"],Similarity(row["id"],basemovie["id"].values[0])))
    distances.sort(key = operator.itemgetter(1))
    
    neighbours = []
    for i in range(k):
        neighbours.append(distances[i][0])
    
    return neighbours

In [65]:
def printNeighbours(K,movie):
    n = getNeighbours(K,movie)
    print(K,"Similar Movies :-")
    display(movies[movies.index.isin(n)][["original_title","director","screenplay"]])

In [66]:
movie = input("Enter Movie to find Similar movies to: ")
K = int(input("How many Simialr movies to be found : "))
printNeighbours(K,movie)

Selected Movie : 3 Idiots
10 Similar Movies :-


,original_title,director,screenplay
2658,Lage Raho Munna Bhai,[Rajkumar Hirani],[Vidhu Vinod Chopra]
4818,Munna Bhai M.B.B.S.,[Rajkumar Hirani],"[Vidhu Vinod Chopra, Rajkumar Hirani, Lajan Joseph, Abbas Tyrewala]"
4926,Rang De Basanti,[Rakeysh Omprakash Mehra],"[Renzil D'Silva, Prasoon Joshi, Rakeysh Omprakash Mehra, Kamlesh Pandey]"
5077,Lage Raho Munna Bhai,[Rajkumar Hirani],"[Vidhu Vinod Chopra, Rajkumar Hirani, Abhijat Joshi]"
5088,Eklavya,[Vidhu Vinod Chopra],"[Vidhu Vinod Chopra, Abhijat Joshi, Swanand Kirkire]"
5341,Hello Darling,[Manoj Tiwari],"[Shabbir Ahmed, Kumaar, Ashiesh Pandit, Sachin Shah, Pankaj Trivedi]"
5478,Toh Baat Pakki!,[Kedar Shinde],"[Shabbir Ahmed, Mayur Puri, Sayeed Qadri, Pramod Sharma, Vibha Singh]"
5633,Ferrari Ki Sawaari,[Rajesh Mapuskar],"[Vidhu Vinod Chopra, Rajesh Mapuskar, Rajkumar Hirani, Ranjeet Bahadur]"
5699,PK,[Rajkumar Hirani],"[Rajkumar Hirani, Abhijat Joshi]"
6129,Sanju,[Rajkumar Hirani],"[Abhiruchi Chand, Rajkumar Hirani, Abhijat Joshi]"


In [5]:
movies_binless = movies[["original_title", "genres", "cast", "director", "screenplay", "keywords"]]

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()

In [7]:
X = movies_binless[["genres", "cast", "director", "screenplay", "keywords"]]


In [8]:
# Use MultiLabelBinarizer to create binary columns for features
genres_bin = pd.DataFrame(mlb.fit_transform(X["genres"]), columns=mlb.classes_, index=X.index)
cast_bin = pd.DataFrame(mlb.fit_transform(X["cast"]), columns=mlb.classes_, index=X.index)
directors_bin = pd.DataFrame(mlb.fit_transform(X["director"]), columns=mlb.classes_, index=X.index)
screenplay_bin = pd.DataFrame(mlb.fit_transform(X["screenplay"]), columns=mlb.classes_, index=X.index)
keywords_bin = pd.DataFrame(mlb.fit_transform(X["keywords"]), columns=mlb.classes_, index=X.index)

In [9]:
# Combine binary columns into a single dataframe
enc_data = pd.concat([genres_bin, cast_bin, directors_bin, screenplay_bin, keywords_bin], axis=1)

In [10]:
enc_data.to_csv("data/encoded_features.csv", sep=",", index=True, index_label="id")

In [159]:
# Fit a KNN model using cosine similarity
from sklearn.neighbors import NearestNeighbors

model_knn = NearestNeighbors(n_neighbors=10, metric="cosine", n_jobs=-1)
model_knn.fit(enc_data)

NearestNeighbors(metric='cosine', n_jobs=-1, n_neighbors=10)

In [210]:
# A function to clean the search string
def clean_string(input_string):
    # Remove characters that are not letters or numbers
    cleaned_string = re.sub(r'[^a-zA-Z0-9]', '', input_string)
    # Remove whitespaces
    cleaned_string = cleaned_string.replace(' ', '')
    # Convert to lowercase
    cleaned_string = cleaned_string.lower()
    return cleaned_string

# Create a function to recommend top 10 movies
def recommend_movies(movie,nn_data,orig_data):
    basemovie = orig_data.iloc[orig_data[orig_data["original_title"].apply(lambda x: clean_string(x)).str.contains(clean_string(movie))].original_title.str.len().sort_values().index[0]].to_frame().T
    movie_index=nn_data[orig_data.original_title==basemovie.original_title.values[0]].index
    _, indices = model_knn.kneighbors(np.array(nn_data.iloc[movie_index]).reshape(1, -1),n_neighbors=10)

    out=orig_data.iloc[indices[0]].iloc[1:, :]
    return out

In [212]:
recommend_movies("spiderman", enc_data, movies)

c:\Users\91916\Desktop\Coding_Trash\WebDev\Data-map-backend\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


,level_0,index,id,original_title,genres,cast,director,screenplay,keywords,genres_bin,cast_bin,director_bin,screenplay_bin,keywords_bin
5,5,5,5,Spider-Man 3,"[Fantasy, Action, Adventure]","[TobeyMaguire, KirstenDunst, JamesFranco, ThomasHadenChurch, TopherGrace]",[Sam Raimi],[Ivan Raimi],"[dualidentity, amnesia, sandstorm, ""loveofoneslife"", forgiveness, spider, wr...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
26,26,26,26,Spider-Man 2,"[Action, Adventure, Fantasy]","[TobeyMaguire, KirstenDunst, JamesFranco, AlfredMolina, RosemaryHarris]",[Sam Raimi],[Alvin Sargent],"[dualidentity, ""loveofoneslife"", pizzaboy, marvelcomic, sequel, superhero, d...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
191,191,191,191,The Wolverine,"[Action, ScienceFiction, Adventure, Fantasy]","[HughJackman, HiroyukiSanada, FamkeJanssen, WillYunLee, TaoOkamoto]",[James Mangold],[Mark Bomback],"[japan, samurai, mutant, worldwari, marvelcomic, superhero, basedoncomicbook...","[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
647,647,647,647,Ghost Rider: Spirit of Vengeance,"[Action, Fantasy, Thriller]","[NicolasCage, CiaránHinds, ViolantePlacido, FergusRiordan, JohnnyWhitworth]",[Mark Neveldine],[Scott M. Gimple],"[monk, easterneurope, skeleton, biker, marvelcomic, superhero, motorcycle, d...","[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
109,109,109,109,Thor: The Dark World,"[Action, Adventure, Fantasy]","[ChrisHemsworth, NataliePortman, TomHiddleston, AnthonyHopkins, ChristopherE...",[Alan Taylor],[Christopher Yost],"[marvelcomic, superhero, basedoncomicbook, hostiletakeover, norsemythology, ...","[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
966,966,966,966,The Shadow,"[Adventure, Fantasy, Action, Thriller, ScienceFiction]","[AlecBaldwin, JohnLone, PenelopeAnnMiller, PeterBoyle, IanMcKellen]",[Russell Mulcahy],[David Koepp],"[newyork, atomicbomb, ladykiller, hypnosis, superhero, basedoncomicbook, rad...","[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 

In [202]:
# Save the KNN model using pickle
import pickle

with open('model.pkl', 'wb') as f:
    pickle.dump(model_knn, f)

In [206]:
movies["original_title"].to_csv("data/movie_titles.csv", index=True, index_label="id")